In [1]:
import copy
import os
import random
import sys

In [2]:
import numpy as np
from tqdm import trange

In [3]:
import time

In [4]:
from build_tree import (build_center_single, build_distribute_four,
                        build_distribute_nine,
                        build_in_center_single_out_center_single,
                        build_in_distribute_four_out_center_single,
                        build_left_center_single_right_center_single,
                        build_up_center_single_down_center_single)
from sampling import sample_attr, sample_attr_avail, sample_rules
from Rule import Rule_Wrapper

In [5]:
from rendering import render_panel

# drawing the image from np.array() file

In [6]:
import cv2

In [75]:
img = render_panel(row_1_1)

In [76]:
# turn on img with new window
cv2.imshow('result', img)
cv2.waitKey(1)

-1

In [77]:
img1 = render_panel(start_node)

In [78]:
# turn on img with new window
cv2.imshow('result1', img1)
cv2.waitKey(1)

-1

In [107]:
# turn off all windows
cv2.destroyAllWindows()

## explore the tree

In [11]:
class args():
    pass

In [12]:
args.num_samples = 5

In [13]:
all_configs = {"center_single": build_center_single(),
                   "distribute_four": build_distribute_four(),
                   "distribute_nine": build_distribute_nine(),
                   "left_center_single_right_center_single": build_left_center_single_right_center_single(),
                   "up_center_single_down_center_single": build_up_center_single_down_center_single(),
                   "in_center_single_out_center_single": build_in_center_single_out_center_single(),
                   "in_distribute_four_out_center_single": build_in_distribute_four_out_center_single()}

In [14]:
# for key in all_configs.keys():
key = "distribute_four"

In [15]:
for k in trange(args.num_samples):
    cont_num = k % 10

100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]


In [16]:
root = all_configs[key]

In [17]:
while True:
    rule_groups = sample_rules()
    new_root = root.prune(rule_groups)
    if new_root is not None:
        break

In [18]:
start_node = new_root.sample()

In [19]:
row_1_1 = copy.deepcopy(start_node)
row_1_1.children[0].children[0].children[0].children

[Entity.0, Entity.1, Entity.2]

In [99]:
first_layout = start_node.children[0].children[0].children[0]
first_layout

Layout.Distribute_Four

In [104]:
first_layout.children[1].type.get_value()

'triangle'

In [35]:
first_position = first_layout.position
first_position

Attribute.Position

In [66]:
first_position.value_idx

array([0, 2, 3])

In [52]:
second_layout = row_1_1.children[0].children[0].children[0]
second_layout

Layout.Distribute_Four

In [72]:
second_layout.children[0].bbox

(0.25, 0.25, 0.5, 0.5)

In [73]:
second_layout.children[1].bbox

(0.75, 0.75, 0.5, 0.5)

In [71]:
second_layout.children[0].bbox, second_layout.children[1].bbox = second_layout.children[1].bbox, second_layout.children[0].bbox

In [53]:
second_position = second_layout.position
second_position

Attribute.Position

In [65]:
second_position.value_idx

array([0, 3, 2])

In [64]:
second_position.set_value_idx(np.array([0, 3, 2]))

**TODO : change the position.value_idx manually**

## Define Rules

In [33]:
rule = Rule_Wrapper(name = 'Progression_new', 
             attr = 'Size',
             param = [1],
             component_idx = 0,
             entity_idxs = (1, 2))

In [33]:
rule = Rule_Wrapper(name = 'Progression', 
             attr = 'Size',
             param = [1, 2],
             component_idx = 0)
rule.sample()

In [34]:
rule

In [35]:
output = rule.apply_rule(row_1_1).

In [98]:
for i, a in enumerate(range(2)):
    print(i, a)

0 0
1 1


## Conditioned Target

In [83]:
row_1_1.children[0].children[0].children[0].children

Layout.Distribute_Four

In [105]:
def set_target_with_condition(AoT, attr, value):
    results = {}
    Components = AoT.children[0].children
    for comp_idx, comp in enumerate(Components):
        Entities = comp.children[0].children
        entity_idxs = []
        for enti_idx, enti in enumerate(Entities):
            if attr == "Type":
                if enti.type.get_value() == value:
                    entity_idxs += [enti_idx]
        results[comp_idx] = entity_idxs
        
    return results

In [106]:
set_target_with_condition(start_node, "Type", "triangle")

{0: [1]}

In [31]:
row_1_1.children[0].children[0].children[0].children[entity_idxs[0]]

Entity.1

In [38]:
second_layout = row_1_1.children[0].children[0].children[0]

In [41]:
second_layout.children[2]

Entity.2

In [27]:
entity_idxs = [1, 2]
for idx in entity_idxs:
    print(idx)

1
2


In [45]:
rule.attr

'Color'

In [62]:
rule.params

[-2, -1, 1, 2]

In [46]:
rule.component_idx

0

In [22]:
rule = rule_groups[0][0]
rule

In [48]:
aot = row_1_1
in_aot = row_1_2

In [49]:
current_layout = aot.children[0].children[rule.component_idx].children[0]
second_aot = copy.deepcopy(in_aot)
second_layout = second_aot.children[0].children[rule.component_idx].children[0]

In [52]:
old_value_level = current_layout.children[0].type.get_value_level()
old_value_level

5

In [14]:
row_1_1 = copy.deepcopy(start_node)
for l in range(len(rule_groups)):
    rule_group = rule_groups[l]
    rule_num_pos = rule_group[0]
    row_1_2 = rule_num_pos.apply_rule(row_1_1)
    row_1_3 = rule_num_pos.apply_rule(row_1_2)
    for i in range(1, len(rule_group)):
        rule = rule_group[i]
        row_1_2 = rule.apply_rule(row_1_1, row_1_2)
    for i in range(1, len(rule_group)):
        rule = rule_group[i]
        row_1_3 = rule.apply_rule(row_1_2, row_1_3)
    if l == 0:
        to_merge = [row_1_1, row_1_2, row_1_3]
    else:
        merge_component(to_merge[1], row_1_2, l)
        merge_component(to_merge[2], row_1_3, l)

In [57]:
rule.first_col

False

In [61]:
rule.value

-1

In [58]:
current_layout.uniformity.get_value()

False

In [59]:
current_layout.children

[Entity.0]

In [55]:
# enforce value consistency
if rule.first_col and not current_layout.uniformity.get_value():
    for entity in current_layout.children:
        entity.type.set_value_level(old_value_level)
for entity in second_layout.children:
    entity.type.set_value_level(old_value_level + rule.value)

In [31]:
a = new_root.sample()

In [32]:
ent_a = a.children[0].children[0].children[0].children[0]

In [99]:
from rendering import render_entity

In [100]:
ent_a

Entity.0

In [101]:
img = render_entity(ent_a)

In [102]:
np.sum(rend_a)

3453477

In [34]:
render_panel(new_root)

AssertionError: 